# Obtencion de datos de la API de Semantic Scholar

In [1]:
import requests
import time
import pandas as pd
import os
import json

In [2]:
# constantes
SEMANTIC_SCHOLAR_API_URL = "https://api.semanticscholar.org/graph/v1"
FIELDS = "paperId,title,abstract,year,authors.name,openAccessPdf"

In [7]:
# directorios
DATA_DIR = "C:\\Users\\alvar\\OneDrive\\Escritorio\\TFM\\data" 
EXISTING_CORPUS_FILENAME = "initial_corpus.csv"
EXISTING_CORPUS_PATH = os.path.join(DATA_DIR,"crudo", EXISTING_CORPUS_FILENAME)

EXPANDED_CORPUS_FILENAME = "expanded_initial_corpus.csv"
EXPANDED_CORPUS_PATH = os.path.join(DATA_DIR,"crudo", EXPANDED_CORPUS_FILENAME)

In [ ]:
# --- Función get_papers Adaptada para este notebook ---
def get_papers(query, limit_per_query=50, existing_ids_set=None):
    """
    Obtiene artículos de Semantic Scholar para una consulta dada,
    evitando IDs ya existentes y filtrando por aquellos con abstract.
    Intenta obtener 'limit_per_query' artículos que cumplan los criterios.
    """
    if existing_ids_set is None:
        existing_ids_set = set()
        
    papers_data_for_query = []
    current_offset = 0
    # Para asegurar obtener 'limit_per_query' artículos después de filtrar,
    # La API tiene un máximo de 10000 resultados por query (offset + limit <= 10000)
    max_api_offset_limit = 9900 # No superar el límite de la API para offset+limit
    
    print(f"\nBuscando hasta {limit_per_query} artículos NUEVOS para: '{query}'...")
    
    while len(papers_data_for_query) < limit_per_query and current_offset < max_api_offset_limit :
        # Pedir en lotes de 100 (máximo de la API)
        batch_api_limit = min(100, max_api_offset_limit - current_offset)
        if batch_api_limit <=0: break

        try:
            params = {
                'query': query,
                'limit': batch_api_limit,
                'offset': current_offset,
                'fields': FIELDS
            }
            response = requests.get(f"{SEMANTIC_SCHOLAR_API_URL}/paper/search", params=params)
            response.raise_for_status()
            api_response_data = response.json()

            if not api_response_data.get('data'):
                print("  No se encontraron más datos de la API para esta consulta.")
                break

            found_in_page = 0
            added_this_page = 0
            for paper_item in api_response_data['data']:
                found_in_page +=1
                paper_id = paper_item.get('paperId')
                # Añadir solo si tiene abstract y el ID no es uno ya guardado
                if paper_item.get('abstract') and paper_id and paper_id not in existing_ids_set:
                    papers_data_for_query.append(paper_item)
                    existing_ids_set.add(paper_id) # Marcar este ID como obtenido
                    added_this_page += 1
                    if len(papers_data_for_query) >= limit_per_query:
                        break 
            
            print(f"  Página API: {found_in_page} artículos. Añadidos (nuevos con abstract): {added_this_page}.")

            current_offset += len(api_response_data['data']) 

            if len(papers_data_for_query) >= limit_per_query:
                print(f"  Alcanzado el límite de {limit_per_query} artículos deseados para '{query}'.")
                break
            
            # Si no hay 'next' o el offset es 0 (a veces la API lo devuelve así), parar
            if api_response_data.get('next', 0) == 0 and current_offset == len(api_response_data['data']): 
                 pass 
            elif api_response_data.get('next', 0) == 0:
                 print("  No hay 'next' página indicada por la API.")
                 break


            # Esperar para respetar los límites de la API
            print(f"  Esperando 3 segundos... (Obtenidos para '{query}': {len(papers_data_for_query)}/{limit_per_query})")
            time.sleep(3) 

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429: 
                print(f"  Error 429 (Too Many Requests). Esperando 60 segundos...")
                time.sleep(60) # Espera más larga
            else:
                print(f"  Error HTTP ({e.response.status_code}) en la petición a la API: {e.response.text}")
                print("  Esperando 10 segundos antes de continuar con la siguiente keyword (si hay)...")
                time.sleep(10)
                break # Salir del bucle while para esta keyword
        except requests.exceptions.RequestException as e:
            print(f"  Error de red en la petición a la API: {e}")
            print("  Esperando 10 segundos antes de continuar con la siguiente keyword (si hay)...")
            time.sleep(10)
            break 
        except Exception as e:
            print(f"  Error inesperado: {e}")
            break

    print(f"Finalizada la búsqueda para '{query}'. Total de artículos NUEVOS con abstract obtenidos: {len(papers_data_for_query)}")
    return papers_data_for_query

In [ ]:
df_existing_corpus = pd.DataFrame()
existing_paper_ids = set() # set para busqueda rápida de IDs existentes

if os.path.exists(EXISTING_CORPUS_PATH):
    print(f"Cargando corpus existente desde: {EXISTING_CORPUS_PATH}")
    try:
        df_existing_corpus = pd.read_csv(EXISTING_CORPUS_PATH)
        if 'paperId' in df_existing_corpus.columns:
            # Asegurarse de que no haya NaNs en paperId antes de convertir a lista y luego a set
            existing_paper_ids = set(df_existing_corpus['paperId'].dropna().astype(str).tolist())
        print(f"Cargados {len(df_existing_corpus)} artículos del corpus existente.")
        print(f"Se encontraron {len(existing_paper_ids)} IDs únicos en el corpus existente.")
    except Exception as e:
        print(f"Error al cargar o procesar el corpus existente: {e}")
        df_existing_corpus = pd.DataFrame() # Resetear si hay error
else:
    print(f"No se encontró el archivo de corpus existente en '{EXISTING_CORPUS_PATH}'. Se creará un corpus nuevo/expandido.")

# Lista de columnas que esperamos tener basadas en la variable FIELDS
expected_cols = [field.split('.')[0] for field in FIELDS.split(',')]

# Si el df existente no está vacío, asegurar que tenga todas las columnas esperadas
if not df_existing_corpus.empty:
    for col in expected_cols:
        if col not in df_existing_corpus.columns:
            df_existing_corpus[col] = None # Añadir columnas faltantes con None
    # Reordenar para que coincida con expected_cols si es necesario
    df_existing_corpus = df_existing_corpus[expected_cols]

Cargando corpus existente desde: C:\Users\alvar\OneDrive\Escritorio\TFM\data\crudo\initial_corpus.csv
Cargados 204 artículos del corpus existente.
Se encontraron 204 IDs únicos en el corpus existente.


In [ ]:

new_search_keywords = [
    "economy business models",
    "mental health support using chatbots",
    "Generative AI in education",
    "sports analytics using AI",
    "sports",
    "deep learning",
    "computer vision",
    "medical image analysis",
    "genomics",
    "climate change",
    "renewable energy",
    "supply chain optimization",
    "economic forecasting",
    "oncology research",
    "cardiovascular diseases",
    "water resource management",
    "robotics in manufacturing",
    "renewable energy systems"
]

# Límite de artículos NUEVOS a intentar obtener por cada keyword
# la función get_papers intentará alcanzar este número de artículos
papers_to_fetch_per_keyword = 50 

all_new_papers_list = []


current_known_ids = existing_paper_ids.copy() # Copiar para no modificar el original 

for keyword in new_search_keywords:
    new_papers_for_keyword = get_papers(keyword, 
                                        limit_per_query=papers_to_fetch_per_keyword, 
                                        existing_ids_set=current_known_ids)
    all_new_papers_list.extend(new_papers_for_keyword)
    
    if keyword != new_search_keywords[-1]: # No esperar después de la última keyword
        print("--- Esperando 5 segundos antes de la siguiente keyword ---")
        time.sleep(5)

print(f"\nProceso de obtención de nuevos artículos finalizado.")
print(f"Total de artículos nuevos potencialmente añadidos (antes de convertir a DF): {len(all_new_papers_list)}")


Buscando hasta 50 artículos NUEVOS para: 'economy business models'...
  Página API: 100 artículos. Añadidos (nuevos con abstract): 49.
  Esperando 3 segundos... (Obtenidos para 'economy business models': 49/50)
  Página API: 2 artículos. Añadidos (nuevos con abstract): 1.
  Alcanzado el límite de 50 artículos deseados para 'economy business models'.
Finalizada la búsqueda para 'economy business models'. Total de artículos NUEVOS con abstract obtenidos: 50
--- Esperando 5 segundos antes de la siguiente keyword ---

Buscando hasta 50 artículos NUEVOS para: 'mental health support using chatbots'...
  Página API: 52 artículos. Añadidos (nuevos con abstract): 50.
  Alcanzado el límite de 50 artículos deseados para 'mental health support using chatbots'.
Finalizada la búsqueda para 'mental health support using chatbots'. Total de artículos NUEVOS con abstract obtenidos: 50
--- Esperando 5 segundos antes de la siguiente keyword ---

Buscando hasta 50 artículos NUEVOS para: 'Generative AI in 

In [ ]:
# crear dataframe con los nuevos articulos
if all_new_papers_list:
    df_new_papers = pd.DataFrame(all_new_papers_list)
    
    # Asegurar que df_new_papers tenga todas las columnas esperadas antes de concatenar
    for col in expected_cols:
        if col not in df_new_papers.columns:
            df_new_papers[col] = None
    df_new_papers = df_new_papers[expected_cols] 

    print(f"Se crearon {len(df_new_papers)} filas a partir de los nuevos artículos recuperados.")
    
    # Concatenar el DataFrame existente con los nuevos artículos
    df_combined = pd.concat([df_existing_corpus, df_new_papers], ignore_index=True)
    print(f"Tamaño del corpus combinado (antes de deduplicación final): {len(df_combined)}")
else:
    print("No se recuperaron artículos nuevos. El corpus combinado será igual al existente.")
    df_combined = df_existing_corpus.copy() # Trabajar con una copia si no hay nada que añadir

# Verificación final de duplicados por 'paperId' en el DataFrame combinado
if 'paperId' in df_combined.columns and not df_combined.empty:
    initial_count = len(df_combined)
    # Asegurarse de que paperId sea string para la deduplicación, por si hay mezclas de tipos
    df_combined['paperId'] = df_combined['paperId'].astype(str)
    df_combined.drop_duplicates(subset=['paperId'], keep='first', inplace=True)
    df_combined.reset_index(drop=True, inplace=True)
    final_count = len(df_combined)
    print(f"Se eliminaron {initial_count - final_count} duplicados en la combinación final.")
    print(f"Tamaño final del corpus expandido: {final_count} artículos únicos.")
else:
    print("El DataFrame combinado está vacío o no tiene 'paperId'. No se realizó deduplicación.")

display(df_combined.head())
display(df_combined.tail())

Se crearon 900 filas a partir de los nuevos artículos recuperados.
Tamaño del corpus combinado (antes de deduplicación final): 1104
Se eliminaron 0 duplicados en la combinación final.
Tamaño final del corpus expandido: 1104 artículos únicos.


,paperId,title,abstract,year,authors,openAccessPdf
0,4dc2617f15847af822d1f89c2e5cca39c8cdb7ad,Effect of a Machine Learning Recommender Syste...,This randomized clinical trial investigates th...,2023,"[{'authorId': '39230104', 'name': 'Jamie M. Fa...",{'url': 'https://jamanetwork.com/journals/jama...
1,9778a564510da05080f978fcff23928ead0f1db9,A Machine Learning Recommender System to Tailo...,Background and Objectives\nNursing homes (NHs)...,2019,"[{'authorId': '2509884', 'name': 'G. Gannod'},...",{'url': 'https://academic.oup.com/gerontologis...
2,6a8a21cab225a428c41e3f8c38e18535f68ffacf,A Machine Learning Recommender System Based on...,Changing and moving toward online shopping has...,2020,"[{'authorId': '2172517066', 'name': 'Delshad M...",{'url': 'https://doi.org/10.22541/au.160897179...
3,fc88d1692a0f53f2821499fa8b8f4d049775585f,Matrix Factorization Collaborative-Based Recom...,Saudi Arabia’s tourism sector has recently sta...,2023,"[{'authorId': '2787898', 'name': 'Reham Alabdu...",{'url': 'https://www.mdpi.com/2076-3417/13/17/...
4,9998dc44714a0721caa671243391c1ed5ecfa222,Smart Crop Recommender System-A Machine Learni...,Machine learning has proven its efficacy in so...,2022,"[{'authorId': '90015202', 'name': 'R. K. Ray'}...","{'url': '', 'status': None, 'license': None, '..."


,paperId,title,abstract,year,authors,openAccessPdf
1099,cefdd9906064c199428645512eb6834dfed72ae3,Optimization of Renewable Energy Resources in ...,This paper proposes the optimization of renewa...,2017,"[{'authorId': '144006876', 'name': 'S. S. Redd...",{'url': 'https://riverpublishers.com/journal/j...
1100,a062fbd621a9821a695b9869630feb6bfb0cf585,PSO-Based Smart Grid Application for Sizing an...,This paper introduces an optimal sizing algori...,2016,"[{'authorId': '144816554', 'name': 'M. A. Moha...",{'url': 'https://journals.plos.org/plosone/art...
1101,61b8a96cf12e961628977066d4c4807dcb526879,Smart Integrated Renewable Energy Systems (SIR...,Technical and economic aspects of the viabilit...,2017,"[{'authorId': '30528974', 'name': 'Zeel Mahesh...",{'url': 'https://www.mdpi.com/1996-1073/10/8/1...
1102,76a6c339e61e04069a35610faee5a523b2509f03,Optimization of Renewable Energy Systems: A Re...,In the contrary of decrease of fossil energy n...,2017,"[{'authorId': '112948910', 'name': 'Diriba Kaj...","{'url': '', 'status': None, 'license': None, '..."
1103,ccb0ca74c441aa95d127d33f736827f4ff0ab761,Optimal configuration for isolated hybrid rene...,The configuration of hybrid energy systems has...,2016,"[{'authorId': '49264644', 'name': 'A. Eltamaly...","{'url': '', 'status': 'CLOSED', 'license': Non..."


In [ ]:
# Guardo el corpus expandido en un csv

if not df_combined.empty:
    try:
        # Asegurarse de que la carpeta de datos exista
        os.makedirs(DATA_DIR, exist_ok=True) 
        
        df_combined.to_csv(EXPANDED_CORPUS_PATH, index=False, encoding='utf-8')
        print(f"\nCorpus expandido guardado exitosamente en: {EXPANDED_CORPUS_PATH}")
    except Exception as e:
        print(f"Error al guardar el corpus expandido: {e}")
else:
    print("El DataFrame combinado está vacío. No se guardó ningún archivo.")


Corpus expandido guardado exitosamente en: C:\Users\alvar\OneDrive\Escritorio\TFM\data\crudo\expanded_initial_corpus.csv
